In [39]:
from os import listdir
from os.path import isfile, join
import os
import re
import pandas as pd
import librosa
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from timeit import default_timer as timer
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sn
import matplotlib.pyplot as plt
import kagglehub

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import regularizers, optimizers
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Activation, MaxPooling1D, Dropout
from tensorflow.keras.utils import plot_model,to_categorical

In [4]:
# Download latest version of the database 
path = kagglehub.dataset_download("vbookshelf/respiratory-sound-database")

print("Path to dataset files:", path)

Path to dataset files: /Users/katherinebeaty/.cache/kagglehub/datasets/vbookshelf/respiratory-sound-database/versions/2


In [31]:
# extract lables for kaggle dataset
# Path to patient diagnosis CSV
csv_path_kaggle = os.path.join(path, "Respiratory_Sound_Database/Respiratory_Sound_Database/patient_diagnosis.csv")

# Load CSV
df_kaggle = pd.read_csv(csv_path_kaggle, header=None, names=["Patient_ID", "Diagnosis"])

# Convert to dictionary {Patient_ID: Diagnosis}
patient_labels_kaggle = dict(zip(df_kaggle["Patient_ID"].astype(str), df_kaggle["Diagnosis"]))

# Display first few mappings
print(list(patient_labels_kaggle.items())[:10])  # Check some values

[('101', 'URTI'), ('102', 'Healthy'), ('103', 'Asthma'), ('104', 'COPD'), ('105', 'URTI'), ('106', 'COPD'), ('107', 'COPD'), ('108', 'LRTI'), ('109', 'COPD'), ('110', 'COPD')]


In [ ]:
# extract labels for Mendeley dataset
path_mendeley = "audioDataMendeley/Audio files"

keep_upper = {"COPD", "BRON"}

# Detect diagnosis text in the filename and format it properly.
def format_diagnosis_from_filename(filename):
    name, ext = os.path.splitext(filename)
    
    # Split on first underscore
    parts = name.split("_", 1)
    if len(parts) > 1:
        prefix, diag_segment = parts[0], parts[1]
        
        # Split the diagnosis segment by commas
        diag_parts = diag_segment.split(",")
        formatted_diag_parts = []
        for part in diag_parts:
            part_clean = part.strip()
            if part_clean.upper() in keep_upper:
                formatted_diag_parts.append(part_clean.upper())
            elif part_clean == "N":
                formatted_diag_parts.append("Healthy")
            else:
                formatted_diag_parts.append(part_clean.title())
        
        # Rejoin with commas
        new_diag_segment = ",".join(formatted_diag_parts)
        new_name = f"{prefix}_{new_diag_segment}"
    else:
        new_name = name
    
    return new_name + ext

# read csv and only take necessary columns
for file in os.listdir(path_mendeley):
    if file.endswith(".wav"):
        old_path = os.path.join(path_mendeley, file)

        if "heart failure" in file.lower():
            os.remove(old_path)
            continue

        new_filename = format_diagnosis_from_filename(file)
        new_path = os.path.join(path_mendeley, new_filename)
        
        # Rename the file
        os.rename(old_path, new_path)

COPD
E W
P R L
42
M
Asthma
E W
P R L
21
F
Asthma
E W
P R M
43
F
Healthy
Healthy
P L L
24
M
COPD
E W
P R L
63
M
COPD
E W
P R L
58
F
Lung Fibrosis
Crep
P L L
76
F
Healthy
Healthy
P R M
21
M
Asthma
E W
P L U
41
F
Pneumonia
Crep
P R M
51
M
Asthma
E W
P R U
49
F
Lung Fibrosis
Crep
P L L
76
F
Healthy
Healthy
P L U
33
M
Healthy
Healthy
P L M
70
M
Asthma And Lung Fibrosis
C
A R M
90
M
Asthma
E W
A R U
46
M
Healthy
Healthy
P R U
68
F
Healthy
Healthy
P L L
53
M
Asthma
E W
P L M
60
M
BRON
Crep
P R L
20
M
Healthy
Healthy
P L M
30
M
Asthma
E W
P R M
53
F
Healthy
Healthy
P R U
36
M
Pneumonia
C
P R U
57
M
Asthma
E W
P L U
45
F
Healthy
Healthy
P L U
52
F
BRON
Crep
P L U
68
F
Healthy
Healthy
P L L
32
M
Healthy
Healthy
P R M
74
M
Healthy
Healthy
P L M
26
M
Asthma
E W
P L M
40
M
Healthy
Healthy
P L L
41
M
Healthy
Healthy
P L U
73
F
Healthy
Healthy
P L M
18
M
Asthma
E W
P R U
24
M
Healthy
Healthy
P R M
50
M
Asthma
E W
P R L
60
F
Asthma
E W
P R M
72
F
Healthy
Healthy
P L M
26
M
Asthma
E W
A R L
40
M
Asthma